In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from statsmodels.tsa.stattools import acf,pacf
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
north_east = pd.read_csv("Brazil Data/northeast.csv")

In [3]:
max_date = north_east['Data'].max()
min_date = north_east['Data'].min()

stations = north_east['station_code'].unique()

print(min_date, max_date)
print(stations)

start_date = '2020-04-30'

2000-05-13 2021-04-30
['A401' 'A402' 'A404' 'A405' 'A307' 'A304' 'A406' 'A203' 'A412' 'A302'
 'A306' 'A410' 'A408' 'A308' 'A409' 'A303' 'A305' 'A407' 'A309' 'A311'
 'A301' 'A310' 'A312' 'A313' 'A417' 'A317' 'A416' 'A320' 'A325' 'A324'
 'A339' 'A316' 'A335' 'A331' 'A341' 'A338' 'A314' 'A322' 'A418' 'A315'
 'A336' 'A204' 'A333' 'A205' 'A330' 'A329' 'A337' 'A323' 'A334' 'A319'
 'A332' 'A321' 'A415' 'A340' 'A327' 'A326' 'A318' 'A413' 'A414' 'A422'
 'A328' 'A351' 'A343' 'A426' 'A347' 'A439' 'A440' 'A359' 'A344' 'A428'
 'A222' 'A438' 'A218' 'A419' 'A434' 'A221' 'A207' 'A423' 'A354' 'A220'
 'A433' 'A346' 'A427' 'A357' 'A225' 'A424' 'A217' 'A237' 'A342' 'A349'
 'A223' 'A348' 'A432' 'A436' 'A238' 'A219' 'A421' 'A442' 'A206' 'A431'
 'A429' 'A356' 'A358' 'A345' 'A420' 'A352' 'A355' 'A435' 'A425' 'A441'
 'A437' 'A430' 'A350' 'A443' 'A224' 'A353' 'A362' 'A363' 'A364' 'A444'
 'A361' 'A447' 'A446' 'A445' 'A360' 'A366' 'A365' 'A367' 'A448' 'A450'
 'A452' 'A449' 'S108' 'A455' 'A454' 'A371' 'A373' 'A451

In [4]:
renamed_columns_en = ['date','hour','total precipitation (mm)','pressao atmosferica ao nivel da estacao (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)','max. temperature in the previous hour (°c)','min. temperature in the previous hour (°c)','dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','relative humidity max. in the previous hour (%)','relative humidity min. in the previous hour (%)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)','wind speed (m/s)','region','state','station','station_code','latitude','longitude','height']

def process_raw(df):
    """
    ! drop index
    ! rename the columns
    ! combine hour and date columns -> transform to date_time
    ! append columns data from all stations to keep the data granularity
    ! remove not useful columns
    ! return table 
    """
    df.drop(['index'],inplace=True, axis=1)
    df.columns = renamed_columns_en
    df['date_time'] = pd.to_datetime(df['date'] + ' ' +  df['hour'])
    df.drop(['date','hour','pressao atmosferica ao nivel da estacao (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)', 'dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)'], inplace=True, axis=1)

    # station_codes = list(df.station_code.unique())
    # df_models = by_code(df,station_codes[0])
    # if len(station_codes)>1:
    #     for i in station_codes[1:]:
    #         df_temp = by_code(df,i)
    #         df_models = pd.concat([df_models,df_temp],axis=1)
    return df

# def by_code(df, station_code):
#     """
#     ! function that filters the dataframe by station_code and rename the columns by the status code
#     """
#     df_station = None
#     df_station = df[df['station_code'] == station_code]
#     df_station.index = df_station.iloc[:,-1]
#     df_station.drop(['station_code','date_time'],axis=1,inplace=True)
#     df_station.columns = list(map(lambda x: station_code+'_'+x, df_station.columns))
    return df_station

def clean_na(df, na_value = -9999):
    """ 
    ! replace Na from dataset
    """
    df = df.replace(to_replace=na_value,value=np.NaN)
    df = df[df.iloc[:,1].first_valid_index():]
    df = df.fillna(method='ffill')
    return df

def make_dataset(stations, start_date, df):
    """
    ! filter raw data from date and stations code
    ! process data
    ! clean na
    """
    df = df[df['Data'] >= start_date]
    df = df[df['station_code'].apply(lambda x: x in stations)]
    df = process_raw(df)
    # df = clean_na(df)
    return df.reset_index()   


In [7]:
north_east_Clean = make_dataset(stations,'2019-04-03', north_east)
north_east_Clean

# the start_date variable I set up produces an empty table

,index,total precipitation (mm),max. temperature in the previous hour (°c),min. temperature in the previous hour (°c),relative humidity max. in the previous hour (%),relative humidity min. in the previous hour (%),wind speed (m/s),region,state,station,station_code,latitude,longitude,height,date_time


In [6]:
north_east_Clean.groupby(by=[north_east_Clean['date_time'].dt.year, 
    north_east_Clean['date_time'].dt.month,
    north_east_Clean['date_time'].dt.day ]).mean()


,,,index,total precipitation (mm),max. temperature in the previous hour (°c),min. temperature in the previous hour (°c),relative humidity max. in the previous hour (%),relative humidity min. in the previous hour (%),wind speed (m/s),latitude,longitude,height
date_time,date_time,date_time,,,,,,,,,,
